In [1]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import pandas as pd
import numpy as np
import blackboard
import knowledge_agent as ka
import time
import train_surrogate_models as tm
import random

In [2]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=blackboard.Blackboard)
ka_rp = run_agent(name='ka_rp', base=ka.KaReactorPhysics_Proxy)
ka_rp1 = run_agent(name='ka_rp1', base=ka.KaReactorPhysics_Proxy)

ka_rp.add_blackboard(bb)
ka_rp1.add_blackboard(bb)

ka_rp.set_attr(core_name='core_1')
ka_rp1.set_attr(core_name='core_2')


ka_rp.connect_writer()
ka_rp1.connect_writer()
ka_rp.write_to_bb()
ka_rp.connect_trigger_event()
ka_rp1.connect_trigger_event()


bb.set_attr(trigger_events={0: {}})
bb.send('trigger', 'message')
time.sleep(2)
ka_rp1.write_to_bb()

print(bb.get_attr('lvl_3'))

print(bb.get_attr('trigger_events'))
ns.shutdown()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:15982 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:15982
INFO [2020-02-21 18:02:16.541957] (blackboard): Agent ka_rp given permission to write
INFO [2020-02-21 18:02:16.557048] (blackboard): Trigger Value: 1.0 From Agent: ka_rp1
INFO [2020-02-21 18:02:16.558644] (blackboard): Trigger Value: 1.0 From Agent: ka_rp
INFO [2020-02-21 18:02:16.558733] (ka_rp1): ka_rp1 triggered
INFO [2020-02-21 18:02:16.560383] (ka_rp): ka_rp triggered
INFO [2020-02-21 18:02:19.560345] (blackboard): Agent ka_rp1 given permission to write
{'core_1': {'reactor_parameters': None, 'xs_set': None}, 'core_2': {'reactor_parameters': None, 'xs_set': None}}
{0: {'ka_rp1': 1.0, 'ka_rp': 1.0}}
NS shut down.


# Building our Multi-Agent Blackbaord System

We start by initializing our multi-agent system using osBrain.
For the basic program, three agents will be required.

* Agent 1: Blackboard
* Agent 2: Knowledge Agent - Neutronics
* Agent 3: Knowledge Agent - Blackboard checker

The blackboard agent will retain all of the optimization results obtained by the neutronics agent on the third level of the blackboard.
Abstract level 3 contains raw data in the form of Pandas Dataframe. 
This data consists of the design variables (height, smear, and plutonium fraction), objectives (keff, void coefficient, doppler coefficient, plutonium fraction), and the weights for each objective.

The neutronics knowledge agent (KA) will run a neutronics optimization for a given set of objective weights.
This is performed using Dakota as the optimization engine, and a surrogate model to obtain the reactor paramters (i.e. our objective functions).
Once an optimal solution has been determined, the neutronics KA writes the associated design variables, objective functions, and weights to the blackboard.

The blackboard checker KA examines abstract level 3 of the blackboard to determine if solutions are close to the desired solution.
If a solution is within some percent of the desired solution, the blackboard check KA will take basic information from abstract level 3 and place it in abstract level 2.

We start off by simply initializing the blackboard and neutronics KA.
This will allow us to see how the Dakota interface operates, and will run 5 Dakota optimizations for five different weighting schemes.

In [ ]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=blackboard.Blackboard)
ka_rp = run_agent(name='ka_rp', base=ka.KaReactorPhysics_Proxy)
ka_rp1 = run_agent(name='ka_rp1', base=ka.KaReactorPhysics_Proxy)

ka_rp.add_blackboard(bb)
ka_rp1.add_blackboard(bb)

ka_rp.connect_writer()
ka_rp.write_to_blackboard()
print(bb.get_attr('lvl_3'))
ka_rp.connect_trigger_event()
ka_rp1.connect_trigger_event()

bb.set_attr(trigger_events={0: {}})
bb.send('trigger', 'message')
time.sleep(2)
print(bb.get_attr('trigger_events'))
ns.shutdown()

In [6]:
ns = run_nameserver()
bb = run_agent(name='blackboard', base=blackboard.Blackboard)
ka_rp = run_agent(name='ka_rp', base=ka.KaReactorPhysics_Proxy)
ka_rp.set_attr(function_evals=50)
ka_rp.add_blackboard(bb)
ka_rp.connect_writer()
ka_bb_lvl2 = run_agent(name='ka_bb_lvl2', base=ka.KaBbLvl2_Proxy)
ka_bb_lvl2.add_blackboard(bb)
ka_bb_lvl2.connect_writer()    


ka_rp.objectives = ['keff', 'void_coeff', 'doppler_coeff', 'pu_content']
ka_rp.design_variables = ['height', 'smear', 'pu_content']
ka_rp.results_path = '/Users/ryanstewart/projects/Dakota_Interface/GA/mabs_results/'

lists = {'a': [0,0,0,0], 'w': [1,0,0,0], 'x': [0,0,1,0], 'y': [0,1,0,0], 'z': [0,0,0,1]}
for wl in lists.values():
    print('Examining Reactor: Core_{}_{}_{}_{}'.format(wl[0],wl[1],wl[2],wl[3]))
    ka_rp.set_attr(weights=wl)
    ka_rp.run_dakota_proxy()
    ka_rp.read_dakota_results()
    ka_rp.write_to_bb()
    ka_bb_lvl2.write_to_bb()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:14579 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:14579
Examining Reactor: Core_0_0_0_0
INFO [2020-02-21 22:59:57.943421] (blackboard): Agent ka_rp given permission to write
INFO [2020-02-21 22:59:58.958749] (blackboard): Agent ka_bb_lvl2 given permission to write
INFO [2020-02-21 22:59:58.967043] (ka_bb_lvl2): 1.9124102787769361
Examining Reactor: Core_1_0_0_0
INFO [2020-02-21 23:00:13.264129] (blackboard): Agent ka_rp given permission to write
INFO [2020-02-21 23:00:14.276396] (blackboard): Agent ka_bb_lvl2 given permission to write
INFO [2020-02-21 23:00:14.283532] (ka_bb_lvl2): 1.9124102787769361
INFO [2020-02-21 23:00:14.285118] (ka_bb_lvl2): 1.9124102787769361
Examining Reactor: Core_0_0_1_0
INFO [2020-02-21 23:00:28.055395] (blackboard): Agent ka_rp given permission to write
INFO [2020-02-21 23:00:29.067473] (blackboard): Agent ka_bb_lvl2 given permission to write
INFO [2020-02-21 23:00:29.075643] (ka_bb_lvl2)

In [5]:
ns.shutdown()

# Data Extraction 

The previous cell built and ran 5 different Dakota optimization runs to examine the effects that the weighting scheme has on the objectives.
The neutronics KA obtained this informatin using a surrogate model to perform the optimization proces.
Results were then written to the blackboard, where they can easily be examined.
Below we print out the optimized design variables along with the associated objective functions.


In [3]:
lvl_3 = bb.get_attr('lvl_3')
for k,v in lvl_3.items():
    print('Reactor: {} \n Height: {} Smear: {} Pu: {} \n keff: {} Void: {}  Doppler: {} \n Weights : ({},{},{},{})'.format(k, 
        v['reactor_parameters']['height'][k], v['reactor_parameters']['smear'][k], v['reactor_parameters']['pu_content'][k], round(v['reactor_parameters']['keff'][k],4), round(v['reactor_parameters']['void'][k],2), round(v['reactor_parameters']['Doppler'][k],4),
        v['reactor_parameters']['w_keff'][k],v['reactor_parameters']['w_void'][k],v['reactor_parameters']['w_dopp'][k],v['reactor_parameters']['w_pu'][k],))
    

Reactor: core_0000 
 Height: 79.8 Smear: 67.05 Pu: 0.6025 
 keff: 1.1702 Void: -64.89  Doppler: -0.4401 
 Weights : (0,0,0,0)
Reactor: core_1000 
 Height: 78.95 Smear: 67.25 Pu: 0.9175 
 keff: 1.2474 Void: -35.97  Doppler: -0.3506 
 Weights : (1,0,0,0)
Reactor: core_0010 
 Height: 51.7 Smear: 50.65 Pu: 0.14 
 keff: 0.7944 Void: -196.49  Doppler: -0.9912 
 Weights : (0,0,1,0)
Reactor: core_0100 
 Height: 51.2 Smear: 50.75 Pu: 0.0575 
 keff: 0.7721 Void: -204.77  Doppler: -1.0157 
 Weights : (0,1,0,0)
Reactor: core_0001 
 Height: 67.15 Smear: 57.0 Pu: 0.0025 
 keff: 0.8809 Void: -168.29  Doppler: -0.8457 
 Weights : (0,0,0,1)


In [4]:
lists = {'a': [1,1,1,1], 'w': [1,2,2,2], 'x': [2,1,1,1], 'y': [3,1,1,1], 'z': [3,2,1,1],
         'b': [0.5,0.2,0.1,2], 'c': [0.4,0.1,0.2,0.3], 'd': [0.6,0.2,0.1,0.1], 'e': [0.3,0.3,0.3,0.1], 'f': [3,2,1,1]}
for wl in lists.values():
    print('Examining Reactor: Core_{}_{}_{}_{}'.format(wl[0],wl[1],wl[2],wl[3]))
    ka_rp.set_attr(weights=wl)
    ka_rp.run_dakota_proxy()
    ka_rp.read_dakota_results()
    ka_rp.write_to_blackboard()

Examining Reactor: Core_1_1_1_1
Examining Reactor: Core_1_2_2_2
Examining Reactor: Core_2_1_1_1
Examining Reactor: Core_3_1_1_1
Examining Reactor: Core_3_2_1_1
Examining Reactor: Core_0.5_0.2_0.1_2
Examining Reactor: Core_0.4_0.1_0.2_0.3
Examining Reactor: Core_0.6_0.2_0.1_0.1
Examining Reactor: Core_0.3_0.3_0.3_0.1
Examining Reactor: Core_3_2_1_1


In [48]:
for x in range(10):
    wl = [round(10*random.random(),4) for i in range(4)]
    print('Examining Reactor: Core_{}_{}_{}_{}'.format(wl[0],wl[1],wl[2],wl[3]))
    ka_rp.set_attr(weights=wl)
    ka_rp.run_dakota_proxy()
    ka_rp.read_dakota_results()
    ka_rp.write_to_blackboard()    

Examining Reactor: Core_8.0707_9.5284_3.4966_2.7676
Examining Reactor: Core_2.5585_1.8056_7.5348_5.4819
Examining Reactor: Core_2.6661_7.6023_9.5919_3.1685
Examining Reactor: Core_5.5613_9.9184_3.9322_5.1531
Examining Reactor: Core_3.8562_2.1938_0.6832_2.7551
Examining Reactor: Core_5.6895_9.6485_7.0977_6.6008
Examining Reactor: Core_5.9168_4.6631_0.6357_4.0691
Examining Reactor: Core_6.9784_4.2185_1.1668_6.4702
Examining Reactor: Core_8.0338_7.9_8.273_4.3106
Examining Reactor: Core_6.2128_3.0202_5.1883_4.7563


In [57]:
ka_rp.set_attr(weights=[1.15,1,1,1])
ka_rp.run_dakota_proxy()
ka_rp.read_dakota_results()
ka_rp.write_to_blackboard()  
temp_des = []
temp_obj = []
lvl_3 = bb.get_attr('lvl_3')
print(lvl_3['core_1.5111']['reactor_parameters']['keff']['core_1.5111'])
for k,v in lvl_3.items():
    temp_des.append((v['reactor_parameters']['w_keff'][k],v['reactor_parameters']['w_void'][k],v['reactor_parameters']['w_dopp'][k],v['reactor_parameters']['w_pu'][k]))
    temp_obj.append((v['reactor_parameters']['keff'][k],v['reactor_parameters']['void'][k],v['reactor_parameters']['Doppler'][k],v['reactor_parameters']['pu_content'][k]))
print(len(temp_des))
sm = tm.Surrogate_Models()
sm.random=0
sm.update_database(temp_des, temp_obj)
model = 'lr'
sm.update_model(model)
sm.optimize_model(model)
sm.predict(model, [[1.15,1,1,1]])

0.7725662865705842
106


array([[   0.94305477, -145.46843639,   -0.76456695,    0.20273155]])

In [ ]:
model = 'ann'
sm.update_model(model)
sm.optimize_model(model)
print(lvl_3['core_1.5111']['reactor_parameters']['keff']['core_1.5111'],lvl_3['core_1.5111']['reactor_parameters']['void']['core_1.5111'],
      lvl_3['core_1.5111']['reactor_parameters']['Doppler']['core_1.5111'],lvl_3['core_1.5111']['reactor_parameters']['pu_content']['core_1.5111'])
print(sm.predict(model, [[1.15,1,1,1]]))
print(sm.models[model]['mse_score'], sm.models[model]['score'])

In [ ]:
1.15,1,1,1
104 - array([[   0.95067783, -142.97894534,   -0.75043104,    0.19088695]])
94 - array([[   0.9458636 , -144.3811169 ,   -0.76500257,    0.21920749]])
84 - array([[   0.93697795, -147.64010498,   -0.7750269 ,    0.19071838]])
74 - array([[   0.92148437, -153.11656489,   -0.7958003 ,    0.16533378]])
64 - array([[   0.9188606 , -153.81344203,   -0.80680371,    0.18924554]])
54 - array([[   0.94344721, -145.17364131,   -0.77106811,    0.22301122]])
44 - array([[   0.92699738, -150.78454251,   -0.79527818,    0.2197479 ]])
34 - array([[   0.93131904, -149.41960065,   -0.7891247 ,    0.20827811]])
5 - array([[   1.00889294, -121.2877934 ,   -0.67933475,    0.41458333]])
